In [1]:
import pandas as pd
import numpy as np

In [2]:
main_df = pd.read_csv('credit_risk/application_train.csv')
test_df = pd.read_csv('credit_risk/application_test.csv')
train_target = main_df[['TARGET', 'SK_ID_CURR']]
test_target = test_df['SK_ID_CURR'].copy()

install_df = pd.read_csv('credit_risk/bureau.csv')
install_joined = install_df.merge(train_target, on='SK_ID_CURR', how='right')

In [3]:
x = pd.read_csv('output/L2_train.csv')

In [6]:
y = pd.DataFrame(index=x['SK_ID_CURR'])